In [1]:
from itertools import product
import numpy as np
from scipy.stats import norm

In [2]:
class LFSR:
    def __init__(self,n_reg, params, states):
        self.n_reg  = n_reg
        self.params = np.array(params, dtype=np.bool)
        self.states = np.array(states, dtype=np.bool)
    
    def set_params(self, params):
        self.params = params
        
    def set_current_state(self, state):
        self.states = state
    
    def one_iteration(self):
        current_output = self.states[0]
        #print("states before", self.states)
        next_last_state = np.logical_xor.reduce(self.params*self.states)
        #print("next last state", next_last_state)
        self.states = np.roll(self.states, -1)
        self.states[-1] = next_last_state
        #print("states after", self.states)
        return current_output
    
    def iterate(self, n):
        outputs = []
        for i in range(n):
            outputs.append(self.one_iteration())
        return np.array(outputs, dtype = np.bool)

class Geffe:
    def __init__(self, lfsr1, lfsr2, lfsr3):
        self.lfsr1 = lfsr1
        self.lfsr2 = lfsr2
        self.lfsr3 = lfsr3
    
    def one_iteration(self):
        x = self.lfsr1.one_iteration()
        y = self.lfsr2.one_iteration()
        s = self.lfsr3.one_iteration()
        return (s*x) ^ ((s^1)* y)
    
    def iterate(self, n):
        outputs = []
        for i in range(n):
            outputs.append(self.one_iteration())
        return np.array(outputs, dtype = np.bool)
    
def find_beta(n):
    return 1.0 / 2.0**n

def find_N_C(p1,p2,alpha,beta):
    a = p1 - p2
    b = np.sqrt(p1*(1-p1))*norm.ppf(1-alpha)+np.sqrt(p2*(1-p2))*norm.ppf(1-beta)
    c = 0
    d = np.sqrt(b**2-4*a*c)
    N1 = np.power((-b + d)/(2*a), 2)
    N2 = np.power((-b - d)/(2*a), 2)
    N = np.maximum(N1,N2)
    C = N*p1 + np.sqrt(N*p1*(1-p1))*norm.ppf(1-alpha)
    return N,C

def check_N_C(N,C, p1, p2, alpha, beta):
    return C - N*p1  - np.sqrt(N*p1*(1-p1))*norm.ppf(1-alpha), C  - N*p2 + np.sqrt(N*p2*(1-p2))*norm.ppf(1-beta)

def all_states_generator(N):
    return product([False,True], repeat=N)

def calculate_R(z,x):
    return np.sum(np.logical_xor(z,x))

def check_master_LFSR(z, generated, LFSR1, approved1, LFSR2, approved2):
    N = len(generated)
    
    
    for state1 in approved_states1:
        LFSR1.set_current_state(state1)
        generated1 = LFSR1.iterate(N)
        ok1 = False
        for state2 in approved_states2:
            LFSR2.set_current_state(state2)
            generated2 = LFSR2.iterate(N)
            ok2 = True
            for i in range(N):
                z_i, gen, gen1, gen2 = z[i], generated[i], generated1[i], generated2[i]
                if gen1 != gen2:
                    if (gen == 1 and gen1 != z_i) or (gen2 != z_i):
                        ok2 = False
                        break
            if ok2 == True:
                ok1 = True;
        

In [3]:
n_lfsr1 = 11
n_lfsr2 = 9
n_lfsr3 = 10

lfsr_p1 = np.array([1,0,1,0,0,0,0,0,0,0,0], dtype=np.bool) # X11 = X2 + X0
lfsr_p2 = np.array([1,1,0,1,1,0,0,0,0], dtype=np.bool)     # X9 = X4 + X3 + X1 + X0 
lfsr_p3 = np.array([1,0,0,1,0,0,0,0,0,0], dtype=np.bool)   # X10 = X3 + X0
alpha   = 0.01
p1      = 1.0/4
p2      = 1.0/2

#var 3
z_str = """01101000101000101110010001100100001000100000010001111100100011001001011011010001011011101010001100010100000010001011111001101010101111011000100011110100110110011011100001010101101010011110000011000100110111011010111101100101010011110001010011001100111110010000000110100011101110001110000010101101000100010100000100011100111011111110010100001000101011111101011001001001010110111111110000010111100111100100110110110001011011111101101010001010100011001100110010011100010010000110110110101101110100010010"""
z = np.array([int(v) for v in z_str],dtype=np.bool)
N = len(z)

LFSR1 = LFSR(len(lfsr_p1), lfsr_p1, [])
LFSR2 = LFSR(len(lfsr_p2), lfsr_p2, [])
LFSR3 = LFSR(len(lfsr_p3), lfsr_p3, [])
g = Geffe(LFSR1, LFSR2, LFSR3)

# FIRST REGISTER

In [4]:
beta1 = find_beta(n_lfsr1)
N1_f,C1 = find_N_C(1.0/4, 1.0/2, 0.01, beta1)
N1 = int(np.ceil(N1_f))
print(check_N_C(N1_f,C1, p1, p2, alpha, beta1))
print("beta=", beta1)
print("N=", N1, "C=", C1)

(-1.7763568394002505e-15, 0.0)
beta= 0.00048828125
N= 113 C= 38.9176580391


In [7]:
states1 = all_states_generator(n_lfsr1)
z1 = z[:N1]
approved_states1 = []
r_statistics_all = []
for state in states1:
    LFSR1.set_current_state(state)
    generated = LFSR1.iterate(N1)
    r_statistics = calculate_R(z1, generated)
    r_statistics_all.append(r_statistics)
    if r_statistics < C1:
        LFSR1.set_current_state(state)
        approved_states1.append((state,LFSR1.iterate(N)))

In [8]:
approved_states1

[((False, False, False, False, False, False, False, True, False, False, False),
  array([False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False, False, False,  True, False,
          True, False, False, False, False, False, False,  True, False,
         False, False,  True, False, False, False, False,  True, False,
          True, False,  True, False,  True, False, False,  True, False,
         False, False, False, False, False, False,  True,  True, False,
          True, False, False, False, False, False,  True,  True,  True,
         False, False,  True, False, False, False,  True,  True, False,
          True,  True,  True, False,  True, False,  True,  True,  True,
         False,  True, False,  True, False, False, False,  True, False,
          True, False, False, False, False,  True, False,  True, False,
         False, False,  True, False, False,  True, False, False, False,
          True, False,  True, False,  True,  True, False

In [9]:
beta2 = find_beta(n_lfsr2)
N2_f,C2 = find_N_C(1.0/4, 1.0/2, 0.01, beta2)
N2 = int(np.ceil(N2_f))
print(check_N_C(N2_f,C2, p1, p2, alpha, beta2))
print("beta=", beta2)
print("N=", N2, "C=", C2)

(1.7763568394002505e-15, 3.5527136788005009e-15)
beta= 0.001953125
N= 97 C= 33.885591793


In [10]:
states2 = all_states_generator(n_lfsr2)
z2 = z[:N2]
approved_states2 = []
r_statistics_all = []
for state in states2:
    LFSR2.set_current_state(state)
    generated = LFSR2.iterate(N2)
    r_statistics = calculate_R(z2, generated)
    r_statistics_all.append(r_statistics)
    if r_statistics < C2:
        LFSR2.set_current_state(state)
        approved_states2.append((state,LFSR2.iterate(N)))

In [11]:
approved_states2

[((True, True, True, True, True, False, False, False, True),
  array([ True,  True,  True,  True,  True, False, False, False,  True,
         False,  True, False, False, False,  True,  True, False,  True,
          True, False, False,  True, False, False, False,  True,  True,
         False, False,  True, False, False,  True,  True,  True, False,
         False, False,  True, False, False, False, False, False,  True,
         False,  True,  True, False,  True,  True,  True,  True, False,
         False, False,  True,  True,  True, False, False, False, False,
         False, False, False, False,  True, False, False, False, False,
          True,  True, False,  True,  True,  True, False,  True, False,
         False,  True, False,  True,  True,  True, False, False,  True,
          True,  True, False, False,  True,  True, False, False, False,
          True,  True,  True,  True, False, False, False, False,  True,
          True, False, False,  True,  True, False,  True,  True,  True,
   

In [16]:
states3 = all_states_generator(n_lfsr3)
all_combinations = list(product(approved_states1,approved_states2))
approved_states3 = []
for state in states3:
    LFSR3.set_current_state(state)
    generated = LFSR3.iterate(N)
    approved = False
    approved_12 = None
    for (state1,generated1),(state2,generated2) in all_combinations:
        ok = True
        for i in range(N):
            z_i, gen, gen1, gen2 = z[i], generated[i], generated1[i], generated2[i]
            if gen1 != gen2:
                if (gen == 1 and gen1 != z_i) or (gen == 0 and gen2 != z_i):
                    ok = False
                    break
        if ok:
            approved = True
            approved_12 = [state1, state2]
            break
    if approved:
        approved_states3.append((approved_12,state))

In [23]:
approved_states3

[([(False, False, False, False, False, False, False, True, False, True, False),
   (True, True, True, True, True, False, False, False, True)],
  (True, False, False, True, False, True, True, False, False, False))]

In [25]:
(found1,found2),found3 = approved_states3[0]

In [26]:
found1

(False, False, False, False, False, False, False, True, False, True, False)

In [27]:
found2

(True, True, True, True, True, False, False, False, True)

In [28]:
found3

(True, False, False, True, False, True, True, False, False, False)

In [30]:
LFSR1.set_current_state(found1)
LFSR2.set_current_state(found2)
LFSR3.set_current_state(found3)

In [31]:
results = g.iterate(N)

In [34]:
np.all(results == z)

True